<a href="https://colab.research.google.com/github/luiz-oliveir/LSTM/blob/main/LSTM_VAE_com_ajustes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import glob
import datetime
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações
row_mark = 740
batch_size = 128
time_step = 1
x_dim = 4
lstm_h_dim = 8
z_dim = 4
epoch_num = 100
threshold = None

# Diretórios
model_dir = "./lstm_vae_model/"
image_dir = "./lstm_vae_images/"
results_dir = 'C:/Users/Augusto-PC/Documents/GitHub/LSTM/Resumo resultados/'
mensal_dir = 'C:/Users/Augusto-PC/Documents/GitHub/LSTM/LSTM_mensal'

# Criar diretórios
for d in [model_dir, image_dir, results_dir, mensal_dir]:
    os.makedirs(d, exist_ok=True)

def split_normalize_data(all_df):
    train_df = all_df[:row_mark]
    test_df = all_df[row_mark:]

    scaler = MinMaxScaler()
    scaler.fit(np.array(all_df)[:, 1:])
    train_scaled = scaler.transform(np.array(train_df)[:, 1:])
    test_scaled = scaler.transform(np.array(test_df)[:, 1:])
    return train_scaled, test_scaled, scaler

def reshape(da):
    return da.reshape(da.shape[0], time_step, da.shape[1]).astype("float32")

def prepare_training_data(data, batch_size=128):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)
    return dataset

def save_monthly_results(predictions, originals, log_px, df_original, file_name):
    """Salva os resultados mensais em diferentes sheets"""
    # Preparar dados
    df_original['Data'] = pd.to_datetime(df_original['Data'])
    df_original['Mes'] = df_original['Data'].dt.month

    # Nome do arquivo
    output_file = os.path.join(mensal_dir, f'analise_mensal_{os.path.basename(file_name)}')

    # Dicionário de meses
    meses = {
        1:'jan', 2:'fev', 3:'mar', 4:'abr', 5:'mai', 6:'jun',
        7:'jul', 8:'ago', 9:'set', 10:'out', 11:'nov', 12:'dez'
    }

    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for mes, nome_mes in meses.items():
            mask_mes = df_original['Mes'] == mes
            if mask_mes.any():
                indices = df_original[mask_mes].index

                df_mes = pd.DataFrame()
                df_mes['Data'] = df_original.loc[indices, 'Data']
                df_mes['Log_Likelihood'] = log_px[indices].flatten()

                # Adicionar predições e valores originais
                for i in range(predictions.shape[2]):
                    df_mes[f'Predição_{i+1}'] = predictions[indices, 0, i]
                    df_mes[f'Original_{i+1}'] = originals[indices, 0, i]

                # Adicionar métricas
                df_mes['Erro_Médio'] = np.mean(np.abs(
                    predictions[indices, 0, :] - originals[indices, 0, :]), axis=1)

                # Estatísticas mensais
                stats = pd.DataFrame({
                    'Métrica': ['Média Original', 'Média Predição', 'Erro Médio'],
                    'Valor': [
                        np.mean(originals[indices]),
                        np.mean(predictions[indices]),
                        np.mean(df_mes['Erro_Médio'])
                    ]
                })

                # Salvar na sheet do mês
                with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
                    df_mes.to_excel(writer, sheet_name=f'{nome_mes}_dados', index=False)
                    stats.to_excel(writer, sheet_name=f'{nome_mes}_stats', index=False)

                print(f"Dados salvos para {nome_mes}")

    print(f"\nAnálise mensal salva em: {output_file}")
    return output_file

def process_file(file_path):
    """Processa um único arquivo"""
    print(f"\nProcessando arquivo: {file_path}")

    try:
        # Carregar dados
        df = pd.read_excel(file_path, parse_dates=['Data'])

        # Normalizar dados
        train_scaled, test_scaled, scaler = split_normalize_data(df)

        # Preparar dataset
        train_dataset = prepare_training_data(train_scaled)

        # Carregar modelo
        model = tf.keras.models.load_model(os.path.join(model_dir, 'lstm_vae_model.h5'))

        # Fazer predições
        predictions = model.predict(reshape(train_scaled))
        originals = reshape(train_scaled)
        log_px = model.predict(reshape(train_scaled))[2]

        # Salvar resultados mensais
        save_monthly_results(predictions, originals, log_px, df, file_path)

        print(f"Processamento concluído para {file_path}")

    except Exception as e:
        print(f"Erro ao processar {file_path}: {str(e)}")

def main():
    """Função principal"""
    # Ler diretório de dados
    try:
        with open('data_path.txt', 'r') as f:
            data_dir = f.read().strip()
    except Exception as e:
        print(f"Erro ao ler data_path.txt: {str(e)}")
        return

    # Procurar arquivos Excel
    excel_files = [f for f in glob.glob(os.path.join(data_dir, "*.xlsx"))
                  if not os.path.basename(f).startswith('~$')]

    if not excel_files:
        print("Nenhum arquivo Excel encontrado!")
        return

    # Processar cada arquivo
    for file in excel_files:
        process_file(file)

if __name__ == "__main__":
    main()
